## If you like the kernel, consider upvoting it and the associated datasets:

https://www.kaggle.com/abhishek/transformers

https://www.kaggle.com/abhishek/sacremoses

https://www.kaggle.com/abhishek/distilbertbaseuncased

In [ ]:
# OOF, Out of Folds에 대한 설명
# https://daewonyoon.tistory.com/287

### Most of the code in this kernel comes directly from:

https://www.kaggle.com/abazdyrev/use-features-oof

Please consider upvoting it!

In [ ]:
# 필요 패키지 설치
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null

In [ ]:
import os
import sys
import glob
import torch

# 트랜스포머 패키지 임포트
sys.path.insert(0, "../input/transformers/transformers-master/")
import transformers
import numpy as np
import pandas as pd
import math

In [ ]:
# n-사이즈의 말뭉치를 생성하는 제너레이터 정의
# 제너레이터란?
#"제너레이터는 반복자(iterator)와 같은 루프의 작용을 컨트롤하기 위해 쓰여지는 특별한 함수 또는 루틴이다. 사실 모든 제너레이터는 반복자이다. 제너레이터는 배열이나 리스트를 리턴하는 함수와 비슷하며, 
#호출을 할 수 있는 파라메터를 가지고 있고, 연속적인 값들을 만들어 낸다. 하지만 한번에 모든 값을 포함한 배열을 만들어서 리턴하는 대신에 
#yield 구문을 이용해 한 번 호출될 때마다 하나의 값만을 리턴하고, 이런 이유로 일반 반복자에 비해 아주 작은 메모리를 필요로 한다. 간단히 얘기하면 제너레이터는 반복자와 같은 역할을 하는 함수이다."
# 즉, 전체 값을 출력하는 대신 특정 부분까지 출력하고, 그 지점을 기억한 뒤 나중에 이어서 출력하는 함수.




def generator_square(nums):
    for i in nums:
        yield i * i


In [ ]:
emp = [1,2,3,4,5]
gen=generator_square(emp)

In [ ]:
# print(next(gen))
# print(next(gen))
# print(next(gen))
# print(next(gen))
# print(next(gen))
# print(next(gen))

In [ ]:

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
# 데이터 전처리 함수

def fetch_vectors(string_list, batch_size=64):
    # inspired by https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
    
    # cuda로 GPU 사용 
    DEVICE = torch.device("cuda")
    
    # 기학습된 distillBERT tokenizer 호출, 트랜스포머 기반 기학습 모델 호출
    tokenizer = transformers.DistilBertTokenizer.from_pretrained("../input/distilbertbaseuncased/")
    model = transformers.DistilBertModel.from_pretrained("../input/distilbertbaseuncased/")
    
    # 모델 학습/활용시 GPU 사용
    model.to(DEVICE)
    
    # 해당 함수의 입력값으로 받은 string_list와 batch_size로 위에 정의한 chunks를 실행하고, 그 출력물로 나온 data로 토큰화 
    fin_features = []
    # chunks 제너레이터로 입력받은 문장을 64글자 단위로 쪼개어 줌. 그 쪼개어진 문장(청크)을 data로 삼고, 이 청크마다 다음 작업을 실행
    # data의 예시는 "I am trying to understand what kinds of places the spam values o"
    
    for data in chunks(string_list, batch_size):
        tokenized = []
        
        
        
      
        
        
        # 글자를 하나씩 쪼개어 x로 취급한 뒤, 
        for x in data:
            # strip으로 스트링 끝부분의 공백을 제거하고, 띄어쓰기 단위로 단어를 list로 만듦(split함수). 그 중 300개 까지의 단어를 가져와 x에 저장
            x = " ".join(x.strip().split()[:300])
            # 이 x를 디스틸 버트 토크나이저 함수로 토큰화 -> [101,1501,102]의 형태로 저장됨
            
            
            # BPE 방식이란?
            # 문장을 subwords로 쪼개고, 그 중 자주 사용되는 뭉치를 unit으로 사용하게끔 하는 방식. 이를 통해 용량을 아낄 수 있다.


            # WPM은 tokenizer의 일종으로, 문장을 쪼개어 token으로 만들어 주는 작업을 수행함
            # 이러한 BPE 방식과 유사하게, Word Piece Model은 언어에 대한 지식이 없어도 빈출하는 substring을 단어로 학습하게 됨. 이러한 방식을 통해 OOV 문제를 효과적으로 처리 가능
            # WPM은 빈도기반 BPE와는 달리 likelihood 기반.

            tok = tokenizer.encode(x, add_special_tokens=True)
            # 토크나이즈 된 결과(tok)의 512번째 토큰까지를 tokenized에 저장
            tokenized.append(tok[:512])
            

        # 최대 길이를 지정하고(padding할 최대 길이)
    
        max_len = 512

        
        # tokenize한 리스트에서 토큰을 하나씩 꺼냄. 최대 길이(512)에서 이 꺼낸 토큰의 길이를 뺀 만큼을 0으로 채워줌
        # post padding 방식임을 알 수 있음. (keras.sequence.pad_sequences 의 padding argument 참조)
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        
        # BERT 모델은 세가지 임베딩을 입력으로 받음 
        #-> 1) 토큰 임베딩(입력 토큰들을 참조한 임베딩) 2) 세그멘트 임베딩(첫번째 문장인지, 두번째 문장인지) 3) 포지션 임베딩(문장 내 토큰의 절대적 위치)
        
        
        # 어텐션마스크를 padding한 결과물에서 0이 아닌(즉 패딩으로 채워넣은 값이 아닌 진짜 토큰 값) 부분에 1로 표시
        attention_mask = np.where(padded != 0, 1, 0)
        
        # padding 한 결과물을 토치의 텐서로 입력해줌.
        input_ids = torch.tensor(padded).to(DEVICE)
        # 어텐션 마스크도 토치 텐서로 입력 
        attention_mask = torch.tensor(attention_mask).to(DEVICE)
        
        # 텐서의 미분값 계산을 지시하는 requires_grad 인자를 일시적으로 off 상태로 만들어주는 구문 (with torch.no_grad())
        # 마지막 히든 스테이트에 model의 결과물을 저장
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)
            
            
        
        # 피처값들을 fin_features 리스트에 덧붙여줌
        features = last_hidden_states[0][:, 0, :].cpu().numpy()
        fin_features.append(features)
    
    # np.vstack 함수로 vertical하게 matrix/vector를 붙여주는 연산
    fin_features = np.vstack(fin_features)
    return fin_features

In [ ]:
df_train = pd.read_csv("../input/google-quest-challenge/train.csv").fillna("none")
df_test = pd.read_csv("../input/google-quest-challenge/test.csv").fillna("none")

sample = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
target_cols = list(sample.drop("qa_id", axis=1).columns)

# fetch_vectors 뜯어 보기

In [ ]:
# print(type(df_train.question_body.values))
# print(df_train.question_body.values[1])

example=chunks(df_train.question_body.values[1],64)

In [ ]:
# 문장을 64글자씩 쪼개어주는 것을 알 수 있음
# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example))

In [ ]:
# cuda로 GPU 사용 
DEVICE = torch.device("cuda")

# 기학습된 distillbert tokenizer 호출, 트랜스포머 기반 모델 호출
tokenizer = transformers.DistilBertTokenizer.from_pretrained("../input/distilbertbaseuncased/")
model = transformers.DistilBertModel.from_pretrained("../input/distilbertbaseuncased/")

# 모델 학습/활용시 GPU 사용
model.to(DEVICE)

In [ ]:
x="expressed words are unshown   "
# print(x)
# print(x.strip())
# print(x.strip().split())
# print(" ".join(x.strip().split()[:300]))


In [ ]:
# BPE 방식이란?
# 문장을 subwords로 쪼개고, 그 중 자주 사용되는 뭉치를 unit으로 사용하게끔 하는 방식. 이를 통해 용량을 아낄 수 있다.


# WPM은 tokenizer의 일종으로, 문장을 쪼개어 token으로 만들어 주는 작업을 수행함
# 이러한 BPE 방식과 유사하게, Word Piece Model은 언어에 대한 지식이 없어도 빈출하는 substring을 단어로 학습하게 됨. 이러한 방식을 통해 OOV 문제를 효과적으로 처리 가능
# WPM은 빈도기반 BPE와는 달리 likelihood 기반.


In [ ]:
data="I am trying to understand what kinds of places the spam values o"
tokenized=[]
fin_features = []
     
for x in data:
    print(x)
    x = " ".join(x.strip().split()[:300])
    print(x)
    tok = tokenizer.encode(x, add_special_tokens=True)
    print(tok)
    tokenized.append(tok[:512])

In [ ]:
x

In [ ]:
tokenized

In [ ]:
max_len = 512

# tokenize한 리스트에서 토큰을 하나씩 꺼냄. 최대 길이(512)에서 이 꺼낸 토큰의 길이를 뺀 만큼을 0으로 채워줌
# post padding 방식임을 알 수 있음. (keras.sequence.pad_sequences 의 padding argument 참조)
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])

In [ ]:
len(tokenized)

In [ ]:
padded

In [ ]:
# 

attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded).to(DEVICE)
attention_mask = torch.tensor(attention_mask).to(DEVICE)

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
type(last_hidden_states)

In [ ]:
last_hidden_states[0][:, 0, :]

In [ ]:
features = last_hidden_states[0][:, 0, :].cpu().numpy()
fin_features.append(features)

In [ ]:
fin_features = np.vstack(fin_features)


In [ ]:
fin_features[0]

In [ ]:
>>> a = np.array([1, 2, 3])
>>> b = np.array([2, 3, 4])
>>> np.vstack((a,b))


### petch_vectors 뜯어보기 끝


# 실행하기

In [ ]:
train_question_body_dense = fetch_vectors(df_train.question_body.values)
train_answer_dense = fetch_vectors(df_train.answer.values)

test_question_body_dense = fetch_vectors(df_test.question_body.values)
test_answer_dense = fetch_vectors(df_test.answer.values)

In [ ]:
train_question_body_dense

In [ ]:
train_question_body_dense.shape

In [ ]:
import os
import re
import gc
import pickle  
import random
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, Lambda
from keras.optimizers import Adam
from keras.callbacks import Callback
from scipy.stats import spearmanr, rankdata
from os.path import join as path_join
from numpy.random import seed
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet

seed(42)
tf.random.set_seed(42)
random.seed(42)

In [ ]:
data_dir = '../input/google-quest-challenge/'
train = pd.read_csv(path_join(data_dir, 'train.csv'))
test = pd.read_csv(path_join(data_dir, 'test.csv'))
print(train.shape, test.shape)
train.head()

In [ ]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

input_columns = ['question_title', 'question_body', 'answer']

In [ ]:
train[targets]

> # Features

In [ ]:
# ^: 일치하지 않는 것, .: 하나의 문자 ---> 즉, 하나의 문자가 아닌 것
# *: 0개 이상 

# 하나의 문자가 아닌 것중에, 다양한 0개 이상의 문자 앞에 나오는...?


find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

In [ ]:
re.findall(find,'d.d.sdasd.asasdasdd')

In [ ]:
urlparse(train['url'][0]).netloc

In [ ]:
re.findall(find,urlparse(train['url'][0]).netloc)[0]

urlparse(train['url'][1]).netloc

re.findall(find,urlparse(train['url'][1]).netloc)

urlparse(train['url'][100]).netloc

re.findall(find,urlparse(train['url'][100]).netloc)

In [ ]:
train['netloc']

In [ ]:
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

In [ ]:
# onehotencoder로 label 달아주기

# 나중에 모델 인풋으로 사용하게 되는데, URL에 있던 링크의 위치와 질문 카테고리를 모델의 input으로 사용하기 위함임.

features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])
ohe = OneHotEncoder()
ohe.fit(merged)

features_train = ohe.transform(train[features]).toarray()
features_test = ohe.transform(test[features]).toarray()

In [ ]:
train[features]

In [ ]:
# print(features_train.shape)

In [ ]:
# universal sentence encoder 모듈 호출 
# The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language tasks.
# tensorflow hub에서 저 인코더를 다운 받아 embed에 저장

module_url = "../input/universalsentenceencoderlarge4/"
embed = hub.load(module_url)

In [ ]:
module_url

In [ ]:
#학습용 embedding, 테스트용 embedding

embeddings_train = {}
embeddings_test = {}

# 인풋으로 들어오는 칼럼의 text(질문 제목, 질문 내용, 대답) 대해,
# print(input_columns)

In [ ]:
text = 'question_title'

In [ ]:
# print(text)

train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()

In [ ]:
curr_train_emb = []
curr_test_emb = []
batch_size = 4
ind = 0
while ind*batch_size < len(train_text):
    curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
    ind += 1

In [ ]:

ind=0
print(train_text[ind*batch_size: (ind + 1)*batch_size])
ind=1
print(train_text[ind*batch_size: (ind + 1)*batch_size])

In [ ]:
embed(train_text[ind*batch_size: (ind + 1)*batch_size])

In [ ]:
# 인덱스 * 배치사이즈가 train_text의 전체 길이보다 작다면,
# curr_train_emb라는 리스트에 train_text의 문장을 4개씩 끊어서 embedding 함수에 넣어 나온 결과를 차곡차곡 쌓아줌


In [ ]:
ind = 0
while ind*batch_size < len(test_text):
    curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
    ind += 1    

embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)

In [ ]:
curr_test_emb

In [ ]:
embeddings_train

In [ ]:
train_text

In [ ]:
input_columns

In [ ]:
for text in input_columns:
    print(text)
    # 물음표와 느낌표를 마침표로 바꿔주고, list로 변환
    train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    #
    curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    while ind*batch_size < len(train_text):
        curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
K.clear_session()
gc.collect()

In [ ]:
# l2 norm, aka 유클리디언 거리
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

# 코사인 거리
cos_dist = lambda x, y: (x*y).sum(axis=1)

In [ ]:
# 질문 제목, 질문 내용, 대답 내용을 임베딩한 벡터간의 거리를 유클리드 및 코사인 거리로 정리해 train 셋으로 만듦
# 테스트로도 만들어 줌.


dist_features_train = np.array([
    l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding'])
]).T


dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T

In [ ]:
[item for k, item in embeddings_train.items()][1]

In [ ]:
# 이 모든 거리값과 피처를 학습과 테스트용 인풋데이터로 뭉쳐줌

X_train = np.hstack([item for k, item in embeddings_train.items()] + [features_train, dist_features_train])
X_test = np.hstack([item for k, item in embeddings_test.items()] + [features_test, dist_features_test])
y_train = train[targets].values

In [ ]:
# 총 3개의 데이터를 쌓아서 인풋으로 넣음
# X_train 역시 3개를 쌓은 건데, embeddings_train은 다음과 같은 입력(train_text)을 embedding한 결과임.
# features_train은 질문의 카테고리, 웹 주소 등을 ohe로 펼쳐 만든 행렬
# dis_features_test는 질문 제목, 질문 내용, 대답 내용을 임베딩한 벡터간의 거리를 유클리드 및 코사인 거리로 계산한 결과

# print(train_text)

In [ ]:
# 이렇게 만든 X_train 에, ['question_title', 'question_body', 'answer'] 이 세가지를 USE 방식으로 임베딩한 결과물까지 붙여 최종 인풋으로 만듦


X_train = np.hstack((X_train, train_question_body_dense, train_answer_dense))
X_test = np.hstack((X_test, test_question_body_dense, test_answer_dense))

In [ ]:
train_question_body_dense

# Modeling

In [ ]:
# Compatible with tensorflow backend
class SpearmanRhoCallback(Callback):
    # Spearman Rho는 순위 상관관계의 비모수적인 추정법
    # 이 대회에서 사용하는 metric임
    # train early stopping을 위해 이 대회의 metric을 기준으로 개선이 일어나지 않으면 학습을 중단시키는 방식으로 callback 사용
    
    def __init__(self, training_data, validation_data, patience, model_name):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        if rho_val >= self.value:
            self.value = rho_val
        else:
            self.bad_epochs += 1
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold" % epoch)
            self.model.stop_training = True
            #self.model.save_weights(self.model_name)
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
y_train.shape[1]

In [ ]:
# 모델 구성부분

def create_model():
    inps = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='elu')(inps)
    x = Dropout(0.2)(x)
    x = Dense(y_train.shape[1], activation='sigmoid')(x)
    model = Model(inputs=inps, outputs=x)
    model.compile(
        optimizer=Adam(lr=1e-4),
        loss=['binary_crossentropy']
    )
    model.summary()
    return model

In [ ]:
all_predictions = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)


for ind, (tr, val) in enumerate(kf.split(X_train)):
    X_tr = X_train[tr]
    y_tr = y_train[tr]
    X_vl = X_train[val]
    y_vl = y_train[val]
    
    model = create_model()
    model.fit(
        X_tr, y_tr, epochs=100, batch_size=32, validation_data=(X_vl, y_vl), verbose=True, 
        callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr), validation_data=(X_vl, y_vl),
                                       patience=5, model_name=f'best_model_batch{ind}.h5')]
    )
    all_predictions.append(model.predict(X_test))
    
model = create_model()
model.fit(X_train, y_train, epochs=33, batch_size=32, verbose=False)
all_predictions.append(model.predict(X_test))

In [ ]:
# Seed 값을 다르게 해놓고, MultiTaskElasticNet으로 학습

kf = KFold(n_splits=5, random_state=2019, shuffle=True)
for ind, (tr, val) in enumerate(kf.split(X_train)):
    X_tr = X_train[tr]
    y_tr = y_train[tr]
    X_vl = X_train[val]
    y_vl = y_train[val]
    
    model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
    model.fit(X_tr, y_tr)
    all_predictions.append(model.predict(X_test))
    
model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
model.fit(X_train, y_train)
all_predictions.append(model.predict(X_test))

In [ ]:
# rankdata : 리스트 내에서 순위를 매겨, 순위 리스트를 출력함

test_preds = np.array([np.array([rankdata(c) for c in p.T]).T for p in all_predictions]).mean(axis=0)
max_val = test_preds.max() + 1
test_preds = test_preds/max_val + 1e-12

In [ ]:
submission = pd.read_csv(path_join(data_dir, 'sample_submission.csv'))
submission[targets] = test_preds
submission.to_csv("submission.csv", index = False)
submission.head()